# Data Gatheration for the recommender

In [268]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import re 
import itertools


#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.preprocessing import MinMaxScaler

# Generate spotify playlists (50)

In [286]:
# normally maximum 1000 songs from a playlist
# But for my playlist no limit!

def get_user_playlist(username, playlist_id, limit=500 ):
    
    attempt = 5
    while attempt > 0:
        try:
            results = sp.user_playlist_tracks(username, playlist_id)
            
            playlist_songs = results['items']
            attempt = 0
        except:
            sleep(0.001)
            print('retrying playlist')
            attempt -= 1
    
    while results['next'] and limit > 0:
        limit -= 50
        try:
            results = sp.next(results)
            playlist_songs.extend(results['items'])
        except:
            print('huuhhh..')
            continue

    return playlist_songs

from time import sleep

def create_dataframe(playlist_id, playlist_name, playlist_songs):
    
    playlist_df = pd.DataFrame()
    for song in playlist_songs:
        if not song or not song['track'] or not song['track']['id']:
                print('unknown song removed')
                playlist_songs.remove(song)
    
    playlist_df['id'] = np.array([song['track']['id'] for song in playlist_songs])
    playlist_df['playlist_id'] = playlist_id
    playlist_df['playlist_name'] = playlist_df.apply(lambda x: playlist_name, axis=1)

    playlist_df['artist_name'] = np.array([song['track']["album"]["artists"][0]["name"] for song in playlist_songs])
    playlist_df['artistid'] = np.array([song['track']["album"]["artists"][0]['external_urls']['spotify'] for song in playlist_songs])
    playlist_df['name'] = np.array([song['track']['name'] for song in playlist_songs])

    playlist_df['username'] = np.array([song['added_by']['id'] for song in playlist_songs])
    playlist_df['added_at'] = np.array([song['added_at'][:10] for song in playlist_songs])
    
    playlist_df['explicit'] = np.array([song['track']['explicit'] for song in playlist_songs])
    playlist_df['duration'] = np.array([song['track']['duration_ms'] for song in playlist_songs])
    playlist_df['popularity'] = np.array([song['track']['popularity'] for song in playlist_songs])

    for i, chunk in enumerate(np.array_split(playlist_df['id'], np.ceil(len(playlist_df) / 100))):
        
        attempt = 5
        while attempt > 0:
            try:
                if i == 0:
                    audio_analysis = sp.audio_features(chunk)
                else:
                    audio_analysis.extend(sp.audio_features(chunk))
                attempt = 0
            except:
                sleep(0.001)
                print('retrying audio feature')
                attempt -= 1


    read_info = True
    for audio_info in audio_analysis:
        if not audio_info:
            read_info = False

    if (read_info):
        playlist_df['danceability'] = np.array([audio_info['danceability'] for audio_info in audio_analysis])
        playlist_df['loudness'] = np.array([audio_info['loudness'] for audio_info in audio_analysis])
        playlist_df['tempo'] = np.array([audio_info['tempo'] for audio_info in audio_analysis])
        playlist_df['acousticness'] = np.array([audio_info['acousticness'] for audio_info in audio_analysis])
        playlist_df['energy'] = np.array([audio_info['energy'] for audio_info in audio_analysis])
        playlist_df['valence'] = np.array([audio_info['valence'] for audio_info in audio_analysis])
        playlist_df['liveness'] = np.array([audio_info['liveness'] for audio_info in audio_analysis])
        playlist_df['loudness'] = np.array([audio_info['loudness'] for audio_info in audio_analysis])
        playlist_df['speechiness'] = np.array([audio_info['speechiness'] for audio_info in audio_analysis])

    return playlist_df


In [315]:
CLIENT_ID = '0e770993e5954933897cfbd6fab1cf34'
CLIENT_SECRET = 'd1ce006764bd44fab215fbc7324d18d7'

import pandas as pd
import numpy as np

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


#sp.user_playlist_tracks('El-Bahramino', 'spotify:playlist:37i9dQZF1DXd9rSDyQguIk')
sp.track('spotify:track:4c0Mgs559lG4sPiEC3XcxY')

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5xLSa7l4IV1gsQfhAMvl0U'},
    'href': 'https://api.spotify.com/v1/artists/5xLSa7l4IV1gsQfhAMvl0U',
    'id': '5xLSa7l4IV1gsQfhAMvl0U',
    'name': 'B.B. King',
    'type': 'artist',
    'uri': 'spotify:artist:5xLSa7l4IV1gsQfhAMvl0U'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AR',
   'AT',
   'AU',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',
   'HT',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JM',
   'JO',
   'JP',
   'KE',
   'KI',
   'KN',
  

In [271]:
for x in sp.categories(limit = 50)['categories']['items']:
    print(x['id'])

toplists
at_home
wellness
radar
mood
pop
hiphop
edm_dance
indie_alt
alternative
rock
classical
jazz
rnb
soul
funk
roots
country
blues
latin
kpop
metal
punk
instrumental
family
student
popculture
decades
party
pride
gaming
workout
chill
romance
dinner
focus
sleep
in_the_car
travel
thirdparty
afro
caribbean
arab
desi


In [293]:

#Get spotify playlists

#playlists_df = pd.DataFrame(columns = ['playlist_id', 'playlist_name', 'artist_name', 'artistid', 'name', 'id', 'username',
#                                              'explicit', 'duration', 'popularity',
#                                             'danceability', 'loudness', 'tempo', 'acousticness', 'energy', 'valence', 'liveness',
#                                             'loudness', 'speechiness'])


categories = [
'rock',
'hiphop',
'decades',
'roots',
'jazz',
'soul',
'blues',
'country',
'funk',
'afro',
'latin',
'caribbean'
]

playlists_df = pd.DataFrame()

for category in categories:
    
    print("Genre: " + category)
    playlists = sp.category_playlists(category)['playlists']
    
    counter = 0

    while playlists:
        #try:
            # Loop through every track in the playlist, extract features and append the features to the playlist df
            for i, playlist in enumerate(playlists['items']):

                if playlist['uri'] == 'spotify:playlist:37i9dQZF1DXec50AjHrNTq':
                    print('skip playlist')
                    continue

                print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))

                # Get playlist songs and add them to dataframe
                if (category == 'blues'):
                    playlist_songs = get_user_playlist('spotify', playlist['uri'], limit=1000)
                else:
                    playlist_songs = get_user_playlist('spotify', playlist['uri'], limit=500)
                    #playlist_songs = sp.user_playlist('spotify', playlist['uri'])['items']
                playlist_songs = create_dataframe(playlist['uri'],  playlist['name'], playlist_songs)
                playlists_df = playlists_df.append(playlist_songs)

                counter +=1
            
            if playlists['next']:
                if counter >= 20:
                    break
                playlists = sp.next(playlists)
            else:
                playlists = None
      #  except:
      #      print('failed, skipping genre')
      #      break


Genre: rock
   1 spotify:playlist:37i9dQZF1DWXRqgorJj26U Rock Classics
   2 spotify:playlist:37i9dQZF1DWYE5MI4mMuii Rock & Roll Party
   3 spotify:playlist:37i9dQZF1DXaKs4phBF9kX 80s Summer Rock
   4 spotify:playlist:37i9dQZF1DX1kznM9KyewU 90s Summer Rock
   5 spotify:playlist:37i9dQZF1DX3oM43CtKnRV 00s Rock Anthems
   6 spotify:playlist:37i9dQZF1DX1rVvRgjX59F 90s Rock Anthems
   7 spotify:playlist:37i9dQZF1DXcF6B6QPhFDv Rock This
   8 spotify:playlist:37i9dQZF1DX11ghcIxjcjE Grunge Forever
   9 spotify:playlist:37i9dQZF1DX0rCrO4CFRfM Acoustic Rock
  10 spotify:playlist:37i9dQZF1DX2UXfvEIZvDK Modern Chill Rock
  11 spotify:playlist:37i9dQZF1DX49jUV2NfGku Rock Solid
  12 spotify:playlist:37i9dQZF1DWWsq4e0rDzty Rock School
  13 spotify:playlist:37i9dQZF1DX525kA8Naqsg Raised on Rock
  14 spotify:playlist:37i9dQZF1DX99DRG9N39X3 10s Rock Anthems
  15 spotify:playlist:37i9dQZF1DWVwr24yj95lH Post-Grunge
  16 spotify:playlist:37i9dQZF1DWZryfp6NSvtz All New Rock
  17 spotify:playlist:37i9dQZF1DX

   4 spotify:playlist:37i9dQZF1DXagUeYbNSnOA Fresh Finds: Latin
   5 spotify:playlist:37i9dQZF1DWZoF06RIo9el Future Hits: Latin
   6 spotify:playlist:37i9dQZF1DXbdrcAZnP3Cy Mexican Party Anthems
   7 spotify:playlist:37i9dQZF1DX1QnNyJOBQBv Latin Pop Hits
   8 spotify:playlist:37i9dQZF1DX4OjfOteYnH8 Urbano Rising
   9 spotify:playlist:37i9dQZF1DX4qKWGR9z0LI Salsa Nation
  10 spotify:playlist:37i9dQZF1DX7Qo2zphj7u3 Hits Románticos
  11 spotify:playlist:37i9dQZF1DWSDautEwAKZB Spotify Singles: Latino
  12 spotify:playlist:37i9dQZF1DX4V5eXk4NKqu Ultimate LatinX Indie
  13 spotify:playlist:37i9dQZF1DXbUMEWUyV9JB Super Cumbias
  14 spotify:playlist:37i9dQZF1DWXmQEAjlxGhi Latin Cardio
  15 spotify:playlist:37i9dQZF1DX7HGyCQ2dcNx This Is J Balvin
  16 spotify:playlist:37i9dQZF1DX8evYJzHX5bn This Is Marc Anthony
  17 spotify:playlist:37i9dQZF1DWYey22ryYM8U Paraíso Tropical
  18 spotify:playlist:37i9dQZF1DX5AVYhCeISA6 Latin Divas
  19 spotify:playlist:37i9dQZF1DWYl3xONiBhif This Is El Alfa
  20 s

In [294]:
playlists_df

,id,playlist_id,playlist_name,artist_name,artistid,name,username,added_at,explicit,duration,popularity,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness
0,0hCB0YR03f6AmQaHbwWDe8,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,Whole Lotta Love - 1990 Remaster,,2020-08-17,False,333893,76,0.412,-11.600,89.740,0.048400,0.902,0.422,0.4050,0.4050
1,2aoo2jlRnM3A0NyLQqMN2f,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Jimi Hendrix,https://open.spotify.com/artist/776Uo845nYHJpN...,All Along the Watchtower,,2020-08-17,False,240800,76,0.438,-6.237,113.253,0.002550,0.805,0.564,0.0842,0.0624
2,08mG3Y1vljYA6bvDt4Wqkj,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,AC/DC,https://open.spotify.com/artist/711MCceyCBcFnz...,Back In Black,,2020-08-17,False,255493,83,0.310,-5.678,188.386,0.011000,0.700,0.763,0.0828,0.0470
3,1RJeiAIwR9pZBgJA8ndZLL,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,The Rolling Stones,https://open.spotify.com/artist/22bE4uQ6baNwSH...,"Paint It, Black - Mono",,2020-08-17,False,204480,68,0.505,-7.384,159.669,0.010300,0.836,0.764,0.2950,0.1010
4,66lOpKgTyFjOrac4S1s94g,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,David Bowie,https://open.spotify.com/artist/0oSGxfWSnnOXhD...,Rebel Rebel - 1999 Remaster,,2020-08-17,False,269360,0,0.676,-7.960,126.341,0.001010,0.920,0.624,0.2480,0.0435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0UHy0scck2g0wJoltLTwDi,spotify:playlist:37i9dQZF1DX9udeSemCofA,Heavy Dub,Prince Far I,https://open.spotify.com/artist/6BkTUVaHHgKUAj...,Dub To Africa,,2020-06-16,False,199973,0,0.755,-11.276,129.815,0.000017,0.439,0.499,0.2540,0.0448
46,7eWuk2l3f7kLxzaiuxBWd3,spotify:playlist:37i9dQZF1DX9udeSemCofA,Heavy Dub,Dub Masters,https://open.spotify.com/artist/5hkl3ikch6ReyP...,Flabba Dub,,2020-07-30,False,228840,19,0.787,-15.047,147.707,0.005780,0.351,0.558,0.1230,0.1290
47,5jYVj4prrAjmXd0mNDaGAt,spotify:playlist:37i9dQZF1DX9udeSemCofA,Heavy Dub,Dub Syndicate,https://open.spotify.com/artist/4KIjA6Q0lA17xu...,Jamaican Proverb,,2020-06-16,False,255533,0,0.809,-7.949,158.492,0.002640,0.600,0.657,0.1080,0.1380
48,7d3adDUXTHu3prwiMxbYv5,spotify:playlist:37i9dQZF1DX9udeSemCofA,Heavy Dub,The Congos,https://open.spotify.com/artist/0nGxKkhmIzyu6b...,Feast Of The Passover,,2020-06-16,False,483813,0,0.747,-8.605,142.905,0.003130,0.490,0.737,0.1370,0.0977


In [295]:


urls = ['spotify:playlist:37i9dQZF1DXd9rSDyQguIk' , 'spotify:playlist:37i9dQZF1DWSKpvyAAcaNZ' ,
       'spotify:playlist:37i9dQZF1DX2iUghHXGIjj', 'spotify:playlist:37i9dQZF1DX9stbPFTxeaB', 'spotify:playlist:37i9dQZF1DXbkKnGZHv1kf'
       , 'spotify:playlist:37i9dQZF1DX4rGCw5bMNp1', 'https://open.spotify.com/playlist/37i9dQZF1DXeaMD6NdSui3?si=oBEbrUxFTCCLj6H_JV6emg',
       'spotify:playlist:37i9dQZF1DWSTHVqvNCwNq', 'spotify:playlist:37i9dQZF1DX4snGSoRjGMW',
       'spotify:playlist:37i9dQZF1DXas3RS9LEsSS', 'spotify:playlist:37i9dQZF1DXeba6s3Tb8Fj']
names = [
 'Blues Classics',
'Blues & Roots Rock',
         'Acoustic Blues',
                'Funky Heavy Bluesy', 'Blues Origins', 'Chicago Blues',
'Lone Star Blues', 'Delta Blues' , 'Soul Blues', 'Latin Blues', 'African Blues']


## Add more blues
for url, name in zip(urls, names):
    print(name)
    playlist_songs = get_user_playlist('spotify', url, limit=500)
    playlist_songs = create_dataframe(url,  name, playlist_songs)
    playlists_df = playlists_df.append(playlist_songs)

Blues Classics
retrying playlist
Blues & Roots Rock
retrying playlist
retrying playlist
retrying audio feature
Acoustic Blues
Funky Heavy Bluesy
Blues Origins
Chicago Blues
Lone Star Blues
Delta Blues
Soul Blues
Latin Blues
African Blues


In [296]:
my_playlists = ['spotify:playlist:6iGacQvayMqTcCeiKRE2zR', 
                'spotify:playlist:2RB9gcVAkFgeD5foKz1DAX', 
                'spotify:playlist:6RzzvekKhU0W3q66dZosP8']

my_playlist_name = 'low down groovin'      

for i, playlist in enumerate(my_playlists):
    print("%4d %s" % (i, my_playlist_name))
        
    playlist_songs = get_user_playlist('El-Bahramino', playlist, limit=500)
    playlist_songs = create_dataframe(playlist, my_playlist_name, playlist_songs)
    playlist_songs['username'] = 'El-Bahramino'  
    playlists_df = playlists_df.append(playlist_songs)        
        
playlists_df

   0 low down groovin
retrying playlist
   1 low down groovin
retrying playlist
   2 low down groovin


,id,playlist_id,playlist_name,artist_name,artistid,name,username,added_at,explicit,duration,popularity,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness
0,0hCB0YR03f6AmQaHbwWDe8,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,Whole Lotta Love - 1990 Remaster,,2020-08-17,False,333893,76,0.412,-11.600,89.740,0.04840,0.902,0.422,0.4050,0.4050
1,2aoo2jlRnM3A0NyLQqMN2f,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Jimi Hendrix,https://open.spotify.com/artist/776Uo845nYHJpN...,All Along the Watchtower,,2020-08-17,False,240800,76,0.438,-6.237,113.253,0.00255,0.805,0.564,0.0842,0.0624
2,08mG3Y1vljYA6bvDt4Wqkj,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,AC/DC,https://open.spotify.com/artist/711MCceyCBcFnz...,Back In Black,,2020-08-17,False,255493,83,0.310,-5.678,188.386,0.01100,0.700,0.763,0.0828,0.0470
3,1RJeiAIwR9pZBgJA8ndZLL,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,The Rolling Stones,https://open.spotify.com/artist/22bE4uQ6baNwSH...,"Paint It, Black - Mono",,2020-08-17,False,204480,68,0.505,-7.384,159.669,0.01030,0.836,0.764,0.2950,0.1010
4,66lOpKgTyFjOrac4S1s94g,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,David Bowie,https://open.spotify.com/artist/0oSGxfWSnnOXhD...,Rebel Rebel - 1999 Remaster,,2020-08-17,False,269360,0,0.676,-7.960,126.341,0.00101,0.920,0.624,0.2480,0.0435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,6n4txcXrc9CXZ5yok7k5tZ,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Skinshape,https://open.spotify.com/artist/1itM5tXaK5THgg...,Afande,El-Bahramino,2021-02-15,False,323163,34,0.758,-6.760,169.876,0.51900,0.843,0.932,0.3530,0.1290
335,6TVs0xWfY5g65DDtr8ermM,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Various Artists,https://open.spotify.com/artist/0LyfQWJT6nXafL...,Cryin' Shame,El-Bahramino,2021-02-15,False,172626,0,0.706,-10.529,91.520,0.98800,0.497,0.543,0.1550,0.0489
336,7bYDwTXJQQ6U9DorPunGb2,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Kiosk,https://open.spotify.com/artist/42o3S5oDd4Q4Yy...,Ghanooneh Kham Shodeh Blues (Bent Rules Blues),El-Bahramino,2021-02-16,False,312080,2,0.567,-9.207,97.009,0.02090,0.705,0.818,0.3170,0.0383
337,0qEJKgKlWpMFcFsZyDLveD,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Levon Helm,https://open.spotify.com/artist/1PWHsS9haruM3A...,Feelin' Good,El-Bahramino,2021-02-23,False,211213,21,0.860,-7.478,109.537,0.45100,0.527,0.786,0.0904,0.0452


In [297]:
playlists_df[playlists_df['playlist_name'] == 'low down groovin']

,id,playlist_id,playlist_name,artist_name,artistid,name,username,added_at,explicit,duration,popularity,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness
0,442ekDJOfQ2Hw1ds3NJvdk,spotify:playlist:6iGacQvayMqTcCeiKRE2zR,low down groovin,Chris Joss,https://open.spotify.com/artist/5a7c4KGYYGhtrv...,Tune Down,El-Bahramino,2020-11-10,False,280590,44,0.774,-11.830,148.032,0.3240,0.540,0.529,0.1090,0.0842
1,2zQl59dZMzwhrmeSBEgiXY,spotify:playlist:6iGacQvayMqTcCeiKRE2zR,low down groovin,Herbie Hancock,https://open.spotify.com/artist/2ZvrvbQNrHKwjT...,Watermelon Man,El-Bahramino,2020-11-10,False,388960,58,0.724,-11.190,146.939,0.2620,0.414,0.719,0.0785,0.0588
2,0JcjupySdUvwYNV7DDkbwU,spotify:playlist:6iGacQvayMqTcCeiKRE2zR,low down groovin,Vulfpeck,https://open.spotify.com/artist/7pXu47GoqSYRaj...,Rango II,El-Bahramino,2020-11-10,False,251366,44,0.761,-11.605,76.346,0.7050,0.252,0.440,0.1990,0.0452
3,4zjE0Vt70Q4vUnedUlIWVo,spotify:playlist:6iGacQvayMqTcCeiKRE2zR,low down groovin,Country Joe & The Fish,https://open.spotify.com/artist/0a63dfrxBQKqh1...,Rockin Round The World,El-Bahramino,2020-11-10,False,294754,19,0.662,-18.282,80.520,0.2360,0.345,0.673,0.0845,0.0706
4,0Jl5bIEve3A4axcjY3EgDZ,spotify:playlist:6iGacQvayMqTcCeiKRE2zR,low down groovin,Joe Cocker,https://open.spotify.com/artist/3pFCERyEiP5xeN...,Feelin' Alright,El-Bahramino,2020-11-10,False,250840,60,0.535,-10.142,90.120,0.6250,0.824,0.895,0.2990,0.0391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,6n4txcXrc9CXZ5yok7k5tZ,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Skinshape,https://open.spotify.com/artist/1itM5tXaK5THgg...,Afande,El-Bahramino,2021-02-15,False,323163,34,0.758,-6.760,169.876,0.5190,0.843,0.932,0.3530,0.1290
335,6TVs0xWfY5g65DDtr8ermM,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Various Artists,https://open.spotify.com/artist/0LyfQWJT6nXafL...,Cryin' Shame,El-Bahramino,2021-02-15,False,172626,0,0.706,-10.529,91.520,0.9880,0.497,0.543,0.1550,0.0489
336,7bYDwTXJQQ6U9DorPunGb2,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Kiosk,https://open.spotify.com/artist/42o3S5oDd4Q4Yy...,Ghanooneh Kham Shodeh Blues (Bent Rules Blues),El-Bahramino,2021-02-16,False,312080,2,0.567,-9.207,97.009,0.0209,0.705,0.818,0.3170,0.0383
337,0qEJKgKlWpMFcFsZyDLveD,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Levon Helm,https://open.spotify.com/artist/1PWHsS9haruM3A...,Feelin' Good,El-Bahramino,2021-02-23,False,211213,21,0.860,-7.478,109.537,0.4510,0.527,0.786,0.0904,0.0452


In [298]:
#Get genres
from tqdm import tqdm
from time import sleep

artist_tmp = playlists_df['artistid'].unique()

artist_genres = pd.DataFrame([], columns=['artistid', 'genres'])

for tmp in tqdm(artist_tmp):   
    attempts = 0
    while attempts < 10:
        try:
            sp_artist = sp.artist(tmp)
            break
        except:
            attempts += 1
            print("Failed, retr")
            sleep(0.1)
            
    artist_genres = artist_genres.append({'artistid' : tmp, 'genres': sp_artist['genres'][0:5]}, ignore_index = True)

  2%|▏         | 104/5937 [00:54<1:25:02,  1.14it/s]

Failed, retr


 20%|█▉        | 1184/5937 [08:37<24:37,  3.22it/s] 

Failed, retr


 58%|█████▊    | 3458/5937 [22:33<22:17,  1.85it/s]  

Failed, retr


100%|██████████| 5937/5937 [34:22<00:00,  2.88it/s]  


In [299]:
playlists_df = playlists_df.merge(artist_genres, on='artistid')
playlists_df

,id,playlist_id,playlist_name,artist_name,artistid,name,username,added_at,explicit,duration,popularity,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness,genres
0,0hCB0YR03f6AmQaHbwWDe8,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,Whole Lotta Love - 1990 Remaster,,2020-08-17,False,333893,76,0.412,-11.600,89.740,0.0484,0.902,0.422,0.4050,0.4050,"[album rock, classic rock, hard rock, rock]"
1,6Vjk8MNXpQpi0F4BefdTyq,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,Kashmir - Remaster,,2020-08-17,False,517125,62,0.483,-8.538,80.576,0.4520,0.615,0.594,0.0512,0.0497,"[album rock, classic rock, hard rock, rock]"
2,78lgmZwycJ3nzsdgmPPGNx,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,Immigrant Song - Remaster,,2020-08-17,False,146250,75,0.564,-10.068,112.937,0.0130,0.932,0.619,0.3490,0.0554,"[album rock, classic rock, hard rock, rock]"
3,4ItljeeAXtHsnsnnQojaO2,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,D'yer Mak'er - Remaster,,2020-08-17,False,262748,61,0.525,-8.560,163.503,0.2620,0.929,0.556,0.0754,0.0784,"[album rock, classic rock, hard rock, rock]"
4,0QwZfbw26QeUoIy82Z2jYp,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,Good Times Bad Times - 1993 Remaster,,2020-08-17,False,166266,67,0.476,-9.192,93.584,0.0382,0.717,0.753,0.0818,0.0949,"[album rock, classic rock, hard rock, rock]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14220,18hcUkipN0XprrF1KOwZUh,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Bosley,https://open.spotify.com/artist/4fFbBXQ3LooiXG...,Neon Magazine,El-Bahramino,2021-01-30,False,314398,35,0.659,-8.652,121.020,0.2110,0.611,0.672,0.0657,0.0602,"[baltimore indie, deep new americana]"
14221,0558raZ6YhmEOf0szO4JGF,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Donald Harrison,https://open.spotify.com/artist/53pHwg1sjhqGk7...,Hu-ta-Nay,El-Bahramino,2021-02-01,False,378226,38,0.654,-14.930,112.112,0.4740,0.411,0.708,0.2180,0.0409,"[jazz saxophone, new orleans jazz]"
14222,7ISLcmRNfvsEAZnuARKJUT,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Los Espiritus,https://open.spotify.com/artist/1UnfU05eCWxrY4...,Todo Bien,El-Bahramino,2021-02-04,False,272780,39,0.578,-10.465,95.503,0.1080,0.799,0.669,0.3300,0.0268,"[argentine alternative rock, argentine indie, ..."
14223,7bYDwTXJQQ6U9DorPunGb2,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Kiosk,https://open.spotify.com/artist/42o3S5oDd4Q4Yy...,Ghanooneh Kham Shodeh Blues (Bent Rules Blues),El-Bahramino,2021-02-16,False,312080,2,0.567,-9.207,97.009,0.0209,0.705,0.818,0.3170,0.0383,"[persian alternative, persian rock]"


In [301]:
#Remove duplicate songs in each playlist (mine)
playlists_df = playlists_df.drop_duplicates(subset=['playlist_name', 'id'])
len(playlists_df)

13650

In [304]:
# Remove songs never accuring atleast twice
counts_songs = playlists_df.groupby(['id'])['id'].transform(len)
playlists_df = playlists_df[counts_songs > 1]
print(len(playlists_df))
playlists_df[playlists_df['username'] == 'El-Bahramino']

2552


,id,playlist_id,playlist_name,artist_name,artistid,name,username,added_at,explicit,duration,popularity,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness,genres
169,14XWXWv5FoCbFzLksawpEe,spotify:playlist:6iGacQvayMqTcCeiKRE2zR,low down groovin,The Doors,https://open.spotify.com/artist/22WZ7M8sxp5THd...,Riders on the Storm,El-Bahramino,2020-11-10,False,434720,72,0.550,-11.729,103.886,0.2500,0.722,0.716,0.110,0.0283,"[acid rock, classic rock, psychedelic rock, rock]"
211,4K1imZQQ0yKtJ40vGmUajS,spotify:playlist:2RB9gcVAkFgeD5foKz1DAX,low down groovin,Bob Dylan,https://open.spotify.com/artist/74ASZWbe4lXaub...,Girl from the North Country,El-Bahramino,2021-01-21,False,220400,65,0.607,-23.455,118.867,0.8430,0.142,0.305,0.121,0.0382,"[classic rock, country rock, folk, folk rock, ..."
237,20OFwXhEXf12DzwXmaV7fj,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Creedence Clearwater Revival,https://open.spotify.com/artist/3IYUhFvPQItj6x...,Bad Moon Rising,El-Bahramino,2021-01-24,False,141600,76,0.508,-5.973,178.946,0.0586,0.774,0.942,0.063,0.0321,"[classic rock, country rock, rock, roots rock,..."
271,7rea8oiI7zkZI5dbigK06f,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Fleetwood Mac,https://open.spotify.com/artist/08GQAI4eElDnRO...,Black Magic Woman - 2018 Remaster,El-Bahramino,2021-01-25,False,172693,0,0.457,-11.138,129.423,0.0516,0.412,0.440,0.275,0.0369,"[album rock, classic rock, mellow gold, rock, ..."
277,1IqFh00G2kvvMm8pRMpehA,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Janis Joplin,https://open.spotify.com/artist/4NgfOZCL9Ml67x...,Me and Bobby McGee,El-Bahramino,2020-11-10,False,271333,71,0.430,-10.960,93.405,0.3030,0.463,0.660,0.152,0.0442,"[blues rock, classic rock, psychedelic rock, r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13905,30A3vEsySPkeXzXeMITvwE,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Tinariwen,https://open.spotify.com/artist/2sf2owtFSCvz2M...,Tiwàyyen,El-Bahramino,2020-11-10,False,224400,32,0.460,-8.709,126.437,0.1460,0.622,0.331,0.115,0.0560,"[afrobeat, afropop, desert blues, malian blues..."
13920,36OpkSbeJVaADUgJynvdht,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Boubacar Traoré,https://open.spotify.com/artist/63Bilw49Uv4s2w...,Hona,El-Bahramino,2020-11-10,False,338746,39,0.429,-10.092,183.624,0.5870,0.455,0.597,0.118,0.0369,"[afropop, malian blues, mande pop, world]"
13921,61AJlCxahP7OhPjYojRvN6,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Boubacar Traoré,https://open.spotify.com/artist/63Bilw49Uv4s2w...,M'Badehou,El-Bahramino,2020-11-10,False,367040,36,0.473,-9.865,125.584,0.8870,0.421,0.727,0.105,0.0315,"[afropop, malian blues, mande pop, world]"
13927,05CF4ATpqROLTQVYABNZ3W,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Rokia Traoré,https://open.spotify.com/artist/6sz0k1q2aEtG5d...,Laidu,El-Bahramino,2020-11-10,False,386973,51,0.702,-15.172,99.747,0.6260,0.269,0.397,0.125,0.0365,"[afropop, malian blues, mande pop, world]"


In [305]:
playlists_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2552 entries, 0 to 13933
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2552 non-null   object 
 1   playlist_id    2552 non-null   object 
 2   playlist_name  2552 non-null   object 
 3   artist_name    2552 non-null   object 
 4   artistid       2552 non-null   object 
 5   name           2552 non-null   object 
 6   username       2552 non-null   object 
 7   added_at       2552 non-null   object 
 8   explicit       2552 non-null   bool   
 9   duration       2552 non-null   int32  
 10  popularity     2552 non-null   int32  
 11  danceability   2514 non-null   float64
 12  loudness       2514 non-null   float64
 13  tempo          2514 non-null   float64
 14  acousticness   2514 non-null   float64
 15  energy         2514 non-null   float64
 16  valence        2514 non-null   float64
 17  liveness       2514 non-null   float64
 18  speechi

In [307]:
#Remove playlists with less than 10 remaining songs

counts_playlist = playlists_df.groupby(['playlist_name'])['playlist_name'].transform(len)
playlists_df = playlists_df[counts_playlist >= 10]
#playlists_df['playlist_name'].value_counts() >= 10
playlists_df

,id,playlist_id,playlist_name,artist_name,artistid,name,username,added_at,explicit,duration,popularity,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness,genres
0,0hCB0YR03f6AmQaHbwWDe8,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,Whole Lotta Love - 1990 Remaster,,2020-08-17,False,333893,76,0.412,-11.600,89.740,0.04840,0.902,0.422,0.4050,0.4050,"[album rock, classic rock, hard rock, rock]"
7,0hCB0YR03f6AmQaHbwWDe8,spotify:playlist:37i9dQZF1DWYE5MI4mMuii,Rock & Roll Party,Led Zeppelin,https://open.spotify.com/artist/36QJpDe2go2Kga...,Whole Lotta Love - 1990 Remaster,,2020-08-21,False,333893,76,0.412,-11.600,89.740,0.04840,0.902,0.422,0.4050,0.4050,"[album rock, classic rock, hard rock, rock]"
14,2aoo2jlRnM3A0NyLQqMN2f,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Jimi Hendrix,https://open.spotify.com/artist/776Uo845nYHJpN...,All Along the Watchtower,,2020-08-17,False,240800,76,0.438,-6.237,113.253,0.00255,0.805,0.564,0.0842,0.0624,"[acid rock, blues rock, classic rock, psychede..."
15,0NWPxcsf5vdjdiFUI8NgkP,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Jimi Hendrix,https://open.spotify.com/artist/776Uo845nYHJpN...,Hey Joe,,2020-08-17,False,210160,69,0.346,-5.695,169.492,0.00603,0.768,0.532,0.0244,0.0377,"[acid rock, blues rock, classic rock, psychede..."
16,0wJoRiX5K5BxlqZTolB2LD,spotify:playlist:37i9dQZF1DWXRqgorJj26U,Rock Classics,Jimi Hendrix,https://open.spotify.com/artist/776Uo845nYHJpN...,Purple Haze,,2020-08-17,False,170813,70,0.533,-5.270,108.900,0.00876,0.905,0.486,0.0698,0.0754,"[acid rock, blues rock, classic rock, psychede..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13905,30A3vEsySPkeXzXeMITvwE,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Tinariwen,https://open.spotify.com/artist/2sf2owtFSCvz2M...,Tiwàyyen,El-Bahramino,2020-11-10,False,224400,32,0.460,-8.709,126.437,0.14600,0.622,0.331,0.1150,0.0560,"[afrobeat, afropop, desert blues, malian blues..."
13920,36OpkSbeJVaADUgJynvdht,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Boubacar Traoré,https://open.spotify.com/artist/63Bilw49Uv4s2w...,Hona,El-Bahramino,2020-11-10,False,338746,39,0.429,-10.092,183.624,0.58700,0.455,0.597,0.1180,0.0369,"[afropop, malian blues, mande pop, world]"
13921,61AJlCxahP7OhPjYojRvN6,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Boubacar Traoré,https://open.spotify.com/artist/63Bilw49Uv4s2w...,M'Badehou,El-Bahramino,2020-11-10,False,367040,36,0.473,-9.865,125.584,0.88700,0.421,0.727,0.1050,0.0315,"[afropop, malian blues, mande pop, world]"
13927,05CF4ATpqROLTQVYABNZ3W,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Rokia Traoré,https://open.spotify.com/artist/6sz0k1q2aEtG5d...,Laidu,El-Bahramino,2020-11-10,False,386973,51,0.702,-15.172,99.747,0.62600,0.269,0.397,0.1250,0.0365,"[afropop, malian blues, mande pop, world]"


In [308]:
playlists_df[playlists_df['playlist_name'] == 'low down groovin']

,id,playlist_id,playlist_name,artist_name,artistid,name,username,added_at,explicit,duration,popularity,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness,genres
169,14XWXWv5FoCbFzLksawpEe,spotify:playlist:6iGacQvayMqTcCeiKRE2zR,low down groovin,The Doors,https://open.spotify.com/artist/22WZ7M8sxp5THd...,Riders on the Storm,El-Bahramino,2020-11-10,False,434720,72,0.550,-11.729,103.886,0.2500,0.722,0.716,0.110,0.0283,"[acid rock, classic rock, psychedelic rock, rock]"
211,4K1imZQQ0yKtJ40vGmUajS,spotify:playlist:2RB9gcVAkFgeD5foKz1DAX,low down groovin,Bob Dylan,https://open.spotify.com/artist/74ASZWbe4lXaub...,Girl from the North Country,El-Bahramino,2021-01-21,False,220400,65,0.607,-23.455,118.867,0.8430,0.142,0.305,0.121,0.0382,"[classic rock, country rock, folk, folk rock, ..."
237,20OFwXhEXf12DzwXmaV7fj,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Creedence Clearwater Revival,https://open.spotify.com/artist/3IYUhFvPQItj6x...,Bad Moon Rising,El-Bahramino,2021-01-24,False,141600,76,0.508,-5.973,178.946,0.0586,0.774,0.942,0.063,0.0321,"[classic rock, country rock, rock, roots rock,..."
271,7rea8oiI7zkZI5dbigK06f,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Fleetwood Mac,https://open.spotify.com/artist/08GQAI4eElDnRO...,Black Magic Woman - 2018 Remaster,El-Bahramino,2021-01-25,False,172693,0,0.457,-11.138,129.423,0.0516,0.412,0.440,0.275,0.0369,"[album rock, classic rock, mellow gold, rock, ..."
277,1IqFh00G2kvvMm8pRMpehA,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Janis Joplin,https://open.spotify.com/artist/4NgfOZCL9Ml67x...,Me and Bobby McGee,El-Bahramino,2020-11-10,False,271333,71,0.430,-10.960,93.405,0.3030,0.463,0.660,0.152,0.0442,"[blues rock, classic rock, psychedelic rock, r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13905,30A3vEsySPkeXzXeMITvwE,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Tinariwen,https://open.spotify.com/artist/2sf2owtFSCvz2M...,Tiwàyyen,El-Bahramino,2020-11-10,False,224400,32,0.460,-8.709,126.437,0.1460,0.622,0.331,0.115,0.0560,"[afrobeat, afropop, desert blues, malian blues..."
13920,36OpkSbeJVaADUgJynvdht,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Boubacar Traoré,https://open.spotify.com/artist/63Bilw49Uv4s2w...,Hona,El-Bahramino,2020-11-10,False,338746,39,0.429,-10.092,183.624,0.5870,0.455,0.597,0.118,0.0369,"[afropop, malian blues, mande pop, world]"
13921,61AJlCxahP7OhPjYojRvN6,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Boubacar Traoré,https://open.spotify.com/artist/63Bilw49Uv4s2w...,M'Badehou,El-Bahramino,2020-11-10,False,367040,36,0.473,-9.865,125.584,0.8870,0.421,0.727,0.105,0.0315,"[afropop, malian blues, mande pop, world]"
13927,05CF4ATpqROLTQVYABNZ3W,spotify:playlist:6RzzvekKhU0W3q66dZosP8,low down groovin,Rokia Traoré,https://open.spotify.com/artist/6sz0k1q2aEtG5d...,Laidu,El-Bahramino,2020-11-10,False,386973,51,0.702,-15.172,99.747,0.6260,0.269,0.397,0.125,0.0365,"[afropop, malian blues, mande pop, world]"


In [317]:
playlists_df.to_csv('./spotify_recom.csv', index=False)